<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv

--2020-04-08 16:45:58--  https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4148456 (4.0M) [text/plain]
Saving to: ‘complaints.csv’

complaints.csv      100%[===================>]   3.96M  --.-KB/s    in 0.07s   

2020-04-08 16:45:58 (58.3 MB/s) - ‘complaints.csv’ saved [4148456/4148456]



In [2]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 7.1MB 6.3MB/s 
     |████████████████████████████████| 378kB 46.3MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=859632 sha256=75f5cebe6d92e48da30b11f54218402c797cca5caf2d4b240d5fd3c97f6ece16
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [0]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,  BaggingClassifier, BaggingRegressor, RandomTreesEmbedding,GradientBoostingClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [97]:
data = pd.read_csv('complaints.csv', sep='\t')
print(data.head(10))
y = data["PRODUCT_ID"]
X = data["cleaned_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

   COMPLAINT_ID  ...                                       cleaned_text
0       3178905  ...  go year . contact advis never took loan . advi...
1       3175952  ...  mail valid debt xx/xx/19 valid receiv , receiv...
2       3174747  ...  xx/xx/xxxx appli receiv onlin loan bluechip fi...
3       3173291  ...  xx/xx/xxxx appli receiv onlin loan . loan amou...
4       3172221  ...  told husband left bill . debt would pay within...
5       3172723  ...  payday loan , becam abl pay . everyon famili c...
6       3171497  ...  within past week , contact oracl financi group...
7       3171674  ...  call work said courthous owe money . got phone...
8       3170784  ...  today xx/xx/xxxx cst contact ks . receiv appro...
9       3171479  ...  cashcal attempt collect fraudul debt . ident s...

[10 rows x 5 columns]


In [0]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import FunctionTransformer

In [99]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
clf1 = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=3)
clf2 = RandomForestClassifier(n_estimators=250, random_state=3)
clf3 = GaussianNB()
clf4 = SVC(class_weight="balanced", random_state =30)
eclf = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3), ('svc', clf4)], voting='hard')

voting = Pipeline([
    ('vect', CountVectorizer( analyzer='word', max_features=360, ngram_range=(1,1))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('reducer', TruncatedSVD(n_components=120)),
    ('clf', eclf),
    ])
voting = voting.fit(X_train, y_train)

In [101]:
predictions = voting.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.70
Recall:   0.69
F1-measure:   0.69
Accuracy:   0.69


Почти 70, посмотрим теперь bagging.

In [0]:
# import itertools
# import numpy as np
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import BaggingClassifier

#np.random.seed(0)

In [0]:
# %matplotlib inline

# import itertools
# import numpy as np

# import seaborn as sns
# import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

# from sklearn import datasets

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier

# from sklearn.ensemble import BaggingClassifier
# from sklearn.model_selection import cross_val_score, train_test_split

# from mlxtend.plotting import plot_learning_curves
# from mlxtend.plotting import plot_decision_regions

# np.random.seed(0)

In [0]:

    
clf1 = DecisionTreeClassifier(criterion='entropy', max_depth=1)
clf2 = KNeighborsClassifier(n_neighbors=1)    

#bagging1 = BaggingClassifier(base_estimator=clf1, n_estimators=10, max_samples=0.8, max_features=0.8)
bagging2 = BaggingClassifier(base_estimator=clf2, n_estimators=100, max_samples=0.8, max_features=0.8)

In [0]:
bag = Pipeline([
    ('vect', CountVectorizer( analyzer='word', max_features=360, ngram_range=(1,1))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('reducer', TruncatedSVD(n_components=120)),
    ('clf', bagging2),
    ])
bag = bag.fit(X_train, y_train)

In [131]:
predictions = bag.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.63
Recall:   0.63
F1-measure:   0.62
Accuracy:   0.62
